In [ ]:
#Preliminary predictivemodel for clothing predictions

In [1]:
import numpy as np
import pandas as pd
import random
import datetime
from sklearn.model_selection import train_test_split
import xgboost
from xgboost import XGBClassifier

In [2]:
#Creating the dataset

#features

"""
Weather features:
1. Temperature (Celcius) (-20 to 25)
2. Humidity (%) (relative) (0 to 100)
3. Probability of Precipitation (%) (0 to 100)
4. Wind speed (mph)

User features:
1. Age (15 to 60)
2. Weight (kg)
3. Sex
4. Height (cm)
5. Fat percentage
6. BMI
"""

#output
"""
output will be in the form of a boolean array with each index presenting different clothing articles
0. short sleve shirt + long pants (always on)
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

'\noutput will be in the form of a boolean array with each index presenting different clothing articles\n0. short sleve shirt + long pants (always on)\n1. Thermal (both top and bottom)\n2. Jacket/Hoodie\n3. light downjacket\n4. Thick down/Winter down\n5. Wind breaker\n6. Umbrella/Raincoat\n'

In [3]:
#smart generator for people's height, weight, age, sex
def peopledatagenerator():
    sex = random.randint(0,1)
    age = random.randint(15, 60)
    #stats from (United states)
    #https://en.wikipedia.org/wiki/Average_human_height_by_country
    #https://en.wikipedia.org/wiki/Human_body_weight
    #male
    if sex == 0:
        height = np.random.normal(loc = 175.3, scale = 6.35)
        weight = np.random.normal(loc = 90.6, scale = 15)
    #female
    else:
        height = np.random.normal(loc = 161.3, scale = 5.59)
        weight = np.random.normal(loc = 77.5, scale = 15)
        
    return sex, age, height, weight




In [4]:
from datetime import date
data = []
currentdate = date.today()
for i in range(100000):
    date = (currentdate - datetime.timedelta(i)).strftime("%d/%m/%Y") #display in day/month/year
    temperature = random.randint(0, 45) - 20
    humidity = random.randint(0, 100)
    precipitation = random.randint(0, 100)
    windspeed = random.randint(0, 40) #mph
    #age = random.randint(15, 60)
    #weight = random.randint(40, 120) #kg
    #height = random.randint(110, 200) #cm
    #sex = random.randint(0,1) # 0:M 1:F
    
    
    sex, age, height, weight = peopledatagenerator()

    #fat percentage formula from
    #https://www.gaiam.com/blogs/discover/how-to-calculate-your-ideal-body-fat-percentage
    
    BMI = weight/(height**2)*10000
    if sex == 0:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 16.2
        
    elif sex == 1:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 5.4
    
    #adjustment for negative fatpercentage
    if fatpercentage < 0:
        fatpercentage = 1
        
    datapoint = (date, temperature, humidity, precipitation, windspeed, age, weight, height, sex, fatpercentage, BMI)
    data.append(datapoint)
    

In [5]:
columnstring = 'date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi'
column = columnstring.split(',')
df = pd.DataFrame(data, columns = column)

In [6]:
df[:100]

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi
0,12/01/2022,-18,73,22,2,31,91.095520,161.949428,1,43.409205,34.732671
1,11/01/2022,-3,72,73,26,53,38.417427,155.289006,1,25.907368,15.931140
2,10/01/2022,22,13,1,28,45,100.198380,175.034504,0,33.395929,32.704941
3,09/01/2022,-20,11,73,32,23,94.266305,179.857515,0,24.058786,29.140655
4,08/01/2022,-15,69,5,24,35,109.691824,162.778719,1,52.327559,41.397966
...,...,...,...,...,...,...,...,...,...,...,...
95,09/10/2021,-13,33,66,35,27,74.834035,176.207897,0,18.932090,24.101742
96,08/10/2021,14,40,77,5,54,81.879046,170.329466,1,40.886822,28.222352
97,07/10/2021,-20,29,94,26,48,82.828973,156.397697,1,46.275239,33.862699
98,06/10/2021,-16,2,66,1,41,70.051476,176.216831,1,31.100966,22.559139


In [7]:
#layers

"""
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

#cold resistance calculation

male_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}


female_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}

# cold resistance categories:
# low: 0 - 24
# mid-low: 25 - 49
# mid-high: 50 - 74
# high: 75 - 99
cold_resistance_scores = {
    'low' : 25,
    'mid-low' : 50,
    'mid-high' : 75,
    'high' : 100
}




In [8]:
#returns cold resistance score
#formula:

#cold_resistance = age_score * gender_age_weight + bmi_score + gender_bmi_weight + gender_fatpercentage_weight
def coldresistance(sex, age, fatpercentage, BMI):
    #female case
    if sex == 1:
        if (age <= 15 or age >= 50):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 16 <= age <= 25:
            age_score = cold_resistance_scores['mid-high']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-low']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 14):
            fatpercentage_score = cold_resistance_scores['low']
        elif (14 < fatpercentage <= 20):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (20 < fatpercentage <= 25): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
      
    #men
    else:
        if (age <= 12 or age >= 50):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 16 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 8):
            fatpercentage_score = cold_resistance_scores['low']
        elif (8 < fatpercentage <= 15):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (15 < fatpercentage <= 24): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
            
    #female case
    if sex == 1:
        cold_resistance = age_score/100*female_weights['age'] + bmi_score/100*female_weights['bmi'] + fatpercentage_score/100*female_weights['fatpercentage']
    
    #male case
    elif sex == 0:
        cold_resistance = age_score/100*male_weights['age'] + bmi_score/100*male_weights['bmi'] + fatpercentage_score/100*male_weights['fatpercentage']
    
    return cold_resistance

In [9]:
df['cold_resistance'] = df.apply(lambda row : coldresistance(row['sex'], row['age'], row['fatpercentage'], row['bmi']), axis = 1)

In [10]:
df.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance
0,12/01/2022,-18,73,22,2,31,91.095520,161.949428,1,43.409205,34.732671,100.0
1,11/01/2022,-3,72,73,26,53,38.417427,155.289006,1,25.907368,15.931140,55.0
2,10/01/2022,22,13,1,28,45,100.198380,175.034504,0,33.395929,32.704941,90.0
3,09/01/2022,-20,11,73,32,23,94.266305,179.857515,0,24.058786,29.140655,80.0
4,08/01/2022,-15,69,5,24,35,109.691824,162.778719,1,52.327559,41.397966,100.0


In [11]:
clothesmap = {
    'thermal' : 0, #(both top and bottom)
    'hoodie' : 1,
    'fleece' : 2,
    'wool' : 3,
    'light_down' : 4,
    'thick_down' : 5,
    'wind_breaker' : 6,
    'umbrella' : 7,
    'winter_boots': 8
}

heatmap = {
    'thermal' : 30, #(both top and bottom)
    'hoodie' : 10,
    'fleece' : 20,
    'wool' : 40,
    'light_down' : 50,
    'thick_down' : 60
}



In [12]:
#rules
"""
1. If temp < 5 : Thermal = True and thickdown = True
2. If temp < 15 : Jacket = True
3. If temp < 10 : lightdown = True
4. If windspeed >= 25 : windbreaker = True
5. If precipitation > 50%: umbrella/raincoast = True


cold resistance categories:
low: 0 - 24
mid-low: 25 - 49
mid-high: 50 - 74
high: 75 - 99
"""



def clothingPredictor(coldresistance, temperature, humidity, precipitation, windspeed):
    #prediction is an array of booleans with each index corresponding to a certain clothing article
    prediction = [False] * len(clothesmap.keys())
    
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
    
    
    #low cold resistance
    if coldresistance <= 25:
        #print('low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        elif temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            
        #if temp > 25 dont put on any additional layers
        else:
            pass
            
    #mid-low cold resistance
    elif 25 < coldresistance <= 50:
        #print('mid- low cr')
        if 20 < temperature <= 25:
            prediction[clothesmap['hoodie']] = True
            
        elif 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 25 dont put on any additional layers
        else:
            pass
        
    #mid-high cold resistance
    elif 50 < coldresistance <= 75:
        #print('mid - high cr')
        if 15 <= temperature <= 20:
            prediction[clothesmap['hoodie']] = True
        
        elif 10 <= temperature < 15:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
        #if temp > 20 dont put on any additional layers
        else:
            pass
        
    #high cold resistance
    elif coldresistance > 75:
        #print('high cr')
        if 10 <= temperature <= 15:
            prediction[clothesmap['hoodie']] = True
            
        elif 5 < temperature < 10:
            prediction[clothesmap['hoodie']] = True
            
        elif 0 < temperature <= 5:
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif -10 < temperature <= 0:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True

            
        elif temperature <= -10:
            prediction[clothesmap['thermal']] = True
            prediction[clothesmap['hoodie']] = True
            prediction[clothesmap['light_down']] = True
            prediction[clothesmap['thick_down']] = True
            #print('yay')
        #if temp > 15 dont put on any additional layers
        else:
            pass

        return prediction

In [13]:
def findlowestheatscore(map):
    lowest = 100000
    for i in map:
        if map[i] < lowest:
            lowest = map[i]
    
    return lowest

#temperature range is from -20 to 25
def predictusingheatscore(temperature, cold_resistance, heatmap, windspeed, precipitation):
    #print(temperature)
    prediction = [False] * len(clothesmap.keys())
    
    #wind breaker and umbrella are independent of cold resistance
    if windspeed >= 25:
        prediction[clothesmap['wind_breaker']] = True
    
    if precipitation >= 50:
        prediction[clothesmap['umbrella']] = True
        
    if precipitation >= 50 and temperature <= 5:
        prediction[clothesmap['winter_boots']] = True
    
    #calulation layers required
    lowest_heatscore = findlowestheatscore(heatmap)

    
    #map using linear equation (-20,300) to (25, 0), x is temperature y is insulation needed
    insulation_calculation = -1 * (300/45) * temperature + (-1) * (300/45)* (-25)
    
    insulation_required = insulation_calculation - cold_resistance
    
    if insulation_required < lowest_heatscore:
        #dont need put on any extra layers if insulation required is lower than the lowest_heatscore
        return prediction
    
    else:
        result = []
        
        while insulation_required > 0:
            smallest_difference = 100000
            best_cloth = ''
            #hardstop at 5 layers
            if len(result) == 6:
                break
            
            for clothes in heatmap:
                #avoid repeats
                if clothes in result: continue
                difference = abs(insulation_required - heatmap[clothes])
                if difference < smallest_difference:
                    smallest_difference = difference
                    best_cloth = clothes
            
            result.append(best_cloth)
            insulation_required = insulation_required - heatmap[best_cloth]
            
        for ite in result:
            prediction[clothesmap[ite]] = True
            
        return prediction
                
                
            
            
        

In [20]:
print(predictusingheatscore(20, 10, heatmap, 25, 40))

[False, True, True, False, False, False, True, False, False]


In [15]:
"""
thermal < fleece/wool < hoodie < light_down < thick_down


add shoes/winter shoes

add more dimensions to calculate the score





"""

'\nthermal < fleece/wool < hoodie < light_down < thick_down\n\n\nadd shoes/winter shoes\n\nadd more dimensions to calculate the score\n\n\n\n\n\n'

In [21]:
for article in clothesmap:
    df[article] = df.apply(lambda row : predictusingheatscore(row['temperature'], row['cold_resistance'], heatmap,\
                                                              row['windspeed'], row['precipitation'])[clothesmap[article]], axis = 1)

In [ ]:
# for article in clothesmap:
#     df[article] = df.apply(lambda row : clothingPredictor(row['cold_resistance'], row['temperature'], row['humidity'],\
#                                                           row['precipitation'], row['windspeed'])[clothesmap[article]], axis = 1)

In [22]:
df.head()

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,...,cold_resistance,thermal,hoodie,fleece,wool,light_down,thick_down,wind_breaker,umbrella,winter_boots
0,12/01/2022,-18,73,22,2,31,91.095520,161.949428,1,43.409205,...,100.0,True,True,False,True,True,True,False,False,False
1,11/01/2022,-3,72,73,26,53,38.417427,155.289006,1,25.907368,...,55.0,False,True,True,False,True,True,True,True,True
2,10/01/2022,22,13,1,28,45,100.198380,175.034504,0,33.395929,...,90.0,False,False,False,False,False,False,True,False,False
3,09/01/2022,-20,11,73,32,23,94.266305,179.857515,0,24.058786,...,80.0,True,True,True,True,True,True,True,True,True
4,08/01/2022,-15,69,5,24,35,109.691824,162.778719,1,52.327559,...,100.0,False,False,True,True,True,True,False,False,False


In [23]:
features = df[['temperature', 'humidity', 'precipitation', 'windspeed', 'cold_resistance', 'age', 'weight', 'height', 'sex', 'fatpercentage', 'bmi', 'cold_resistance' ]]

In [24]:
features.head()

,temperature,humidity,precipitation,windspeed,cold_resistance,age,weight,height,sex,fatpercentage,bmi,cold_resistance
0,-18,73,22,2,100.0,31,91.095520,161.949428,1,43.409205,34.732671,100.0
1,-3,72,73,26,55.0,53,38.417427,155.289006,1,25.907368,15.931140,55.0
2,22,13,1,28,90.0,45,100.198380,175.034504,0,33.395929,32.704941,90.0
3,-20,11,73,32,80.0,23,94.266305,179.857515,0,24.058786,29.140655,80.0
4,-15,69,5,24,100.0,35,109.691824,162.778719,1,52.327559,41.397966,100.0


In [ ]:
features.to_csv('features', index = False)

In [ ]:
output = df[['thermal', 'hoodie', 'light_down', 'thick_down', 'wind_breaker', 'umbrella']]

In [ ]:
features.to_csv('features', index = False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, output, test_size=0.25, random_state=10)

In [ ]:
y_train

In [ ]:
model = XGBClassifier()
model.fit_transform(X_train, y_train)

In [ ]:
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, X_train, num_round)